In [1]:
from src.kr6 import KR6
import modern_robotics as mr
import numpy as np
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes
from src.dh import DHFrame
import src.tools as tools
import sympy as sp


In [2]:

# r1, r2, r3, r4, a1, a2 = 0, 0.400, 0.455, 10**(-3)*np.sqrt(420**2+35**2), 0.025, 0.035
# køddevinkel = np.arctan2(35, 420)

T_SB = np.array([[0,  1,   0, 0],
                 [0,  0,  -1, -(0.420+0.025+0.080)],
                 [-1, 0,   0, 0.89],
                 [0,  0,   0, 1]])

#print('forventer at P_W er: \n', tools.pointFromT(tools.makeT_SW(T_SB)))

theta = np.pi/180*np.array([5, -39, 70, 55, 9, 12])

kr6 = KR6(theta)

thetalists = np.around(kr6.aIK_thetaLists,2)

thetalists*180/np.pi



array([[   5.15662016,   23.4912696 ,  -60.73352628],
       [   5.15662016,  -38.96113007,   69.90085101],
       [-174.75212751,  157.56339366,   56.72282172],
       [-174.75212751, -143.23944878,  -46.9825392 ]])

In [3]:
T = kr6.Tsb_poe
# print(T)

theta4_i, theta4_ii= tools.inverseKinematicsTheta456(thetalists, T)

# test = R_S4@R_45@ R_56@ R_6B
# test.simplify()

theta4_ii*(180/np.pi)




TypeError: cannot unpack non-iterable numpy.float64 object

In [ ]:
config_fasit = np.array([0,-np.pi/4, np.pi/6, 0,0,0])
config2 = np.array([0, -np.pi/2, np.pi/2, 0, 0, 0])
config3 = np.array([0, np.pi/2, 0, 0, 0, 0])





# kr6.add_FK_to_viewer(viewer)

# print(f"Number of iteration: {kr6.IK[3]}")

# kr6.add_nIK_to_viewer(viewer)
viewer = Viewer(background="white")
viewer.add(Grid())
KR6(config_fasit).add_FK_to_viewer(viewer)


C:\Users\test\anaconda3\envs\tpk4170_venv\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

c:\Users\test\git\TPK4170_project-1\src\tools.py:177: RuntimeWarning: invalid value encountered in sqrt
  s3_pos = np.sqrt(1-c3**2)
c:\Users\test\git\TPK4170_project-1\src\tools.py:178: RuntimeWarning: invalid value encountered in sqrt
  s3_neg = -np.sqrt(1-c3**2)
